# Postprocessing a Feed-Forward NN Trained for the Perfectly Stirred Reactor Case

In [1]:
### Loading Libraries

import os
import sys
import numpy                              as np
import pandas                             as pd

import matplotlib
from matplotlib                           import pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

In [2]:
### Input Data

ROMNetFldr    = WORKSPACE_PATH + '/ROMNet/romnet/'
PathToRunFld  = WORKSPACE_PATH + '/ROMNet/PSR_10Cases/FNN-SourceTerms/Deterministic/Run_1/'
#PathToRunFld  = WORKSPACE_PATH + '/ROMNet/MSD_10Cases/FNN-SourceTerms/MCDropOut/Run_1/'
#PathToRunFld  = WORKSPACE_PATH + '/ROMNet/MSD_10Cases/FNN-SourceTerms/BayesByBackprop/Run_1/'
#PathToRunFld  = WORKSPACE_PATH + '/ROMNet/MSD_10Cases/FNN-SourceTerms/HamiltonianMC/Run_1/'

TrainingCases = [0]#[0,2,4,6,8]
TestCases     = [0]#[0,2]

NSamples      = 1

NPerRestTrain = 2000
tStratch      = 1.
NtTrain       = NPerRestTrain*2
SOLVER        = 'BDF'
rtol          = 1.e-12 #1.e-15

In [3]:
print("\n[ROMNet]: Reading Input File from: ", PathToRunFld)
sys.path.insert(0, PathToRunFld)

print("\n[ROMNet]: Keep Loading Modules and Functions...")
from ROMNet_Input import inputdata

print("\n[ROMNet]: Initializing Input ...")
InputData                 = inputdata(WORKSPACE_PATH, ROMNetFldr)

InputData.PathToRunFld    = PathToRunFld

InputData.PathToFigFld    = InputData.PathToRunFld+'/Figures/'
print("\n[ROMNet]: Final Figures can be Found here: " + InputData.PathToFigFld)

InputData.PathToParamsFld = InputData.PathToRunFld+'/Params/'
print("\n[ROMNet]: Final Parameters can be Found here: " + InputData.PathToParamsFld)

InputData.TBCheckpointFldr = None


[ROMNet]: Reading Input File from:  /Users/sventuri/WORKSPACE//ROMNet/PSR_10Cases/FNN-SourceTerms/Deterministic/Run_1/

[ROMNet]: Keep Loading Modules and Functions...

[ROMNet]: Initializing Input ...

[ROMNet]: Final Figures can be Found here: /Users/sventuri/WORKSPACE//ROMNet/PSR_10Cases/FNN-SourceTerms/Deterministic/Run_1//Figures/

[ROMNet]: Final Parameters can be Found here: /Users/sventuri/WORKSPACE//ROMNet/PSR_10Cases/FNN-SourceTerms/Deterministic/Run_1//Params/


In [4]:
### Loading Surrogate

SurrogateType = InputData.SurrogateType
if (SurrogateType == 'FNN-SourceTerms'):
    SurrogateType = 'FNN'

sys.path.insert(0, InputData.ROMNetFldr  + '/src/Model/' + SurrogateType + '/' + InputData.ProbApproach + '/')
from Model import model

InputData.TrainIntFlg = 0

NN = model(InputData, InputData.PathToRunFld, None, None)

NN.load_params(InputData) 



# ModelFile      = InputData.PathToRunFld    + '/NNModel'
# Model          = keras.models.load_model(ModelFile, custom_objects={InputData.LossFunction: None})
# Model.summary()

# MCFile         = InputData.PathToParamsFld + "/ModelCheckpoint/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(MCFile)
# latest         = train.latest_checkpoint(checkpoint_dir)
# Model.load_weights(latest)

[ROMNet]:   Defining ML Model from Scratch
[ROMNet]:   Compiling ML Model with Loss and Optimizer
[ROMNet]:   Saving ML Model in File: /Users/sventuri/WORKSPACE//ROMNet/PSR_10Cases/FNN-SourceTerms/Deterministic/Run_1//NNModel
INFO:tensorflow:Assets written to: /Users/sventuri/WORKSPACE//ROMNet/PSR_10Cases/FNN-SourceTerms/Deterministic/Run_1//NNModel/assets
[ROMNet]:   Summarizing ML Model Structure:
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
normalization (Normalization (None, 3)                 7         
_________________________________________________________________
NN_SPC_1_HL1 (Dense)         (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)               

In [5]:
### Evaluating on Training Data

FileName  = InputData.PathToTrainDataFld + '/' + InputData.InputFile
InputDF   = pd.read_csv(FileName, header=0)
Input     = InputDF[InputData.InputVars].to_numpy()
VarNames  = list(InputDF.columns)


FileName  = InputData.PathToTrainDataFld + '/' + InputData.OutputFile
OutputDF  = pd.read_csv(FileName, header=0)
Output    = OutputDF[InputData.OutputVars].to_numpy()

In [13]:
# Plotting Sources vs States

fig  = plt.figure(figsize=(16,12))
ax   = plt.axes(projection='3d')
ax.plot3D(InputDF[InputData.InputVars[0]], InputDF[InputData.InputVars[1]], OutputDF[InputData.OutputVars[0]], 'ok',  linewidth=2)
ax.set_xlabel(InputData.InputVars[0])
ax.set_ylabel(InputData.InputVars[1])
ax.set_zlabel(InputData.OutputVars[0])

fig  = plt.figure(figsize=(16,12))
ax   = plt.axes(projection='3d')
ax.plot3D(InputDF[InputData.InputVars[0]], InputDF[InputData.InputVars[1]], OutputDF[InputData.OutputVars[1]], 'ok',  linewidth=2)
ax.set_xlabel(InputData.InputVars[0])
ax.set_ylabel(InputData.InputVars[1])
ax.set_zlabel(InputData.OutputVars[1])

In [14]:
Ny   = len(InputData.OutputVars)
ax   = []
for iy in range(Ny):
    fig    = plt.figure(figsize=(16,12))
    ax.append(plt.axes())
    xMin_  = np.amin(Output[:,iy])
    xMax_  = np.amax(Output[:,iy])
    ax[iy].plot([xMin_,xMax_],[xMin_,xMax_],'k-')
    ax[iy].set_xlabel('Data, '+InputData.OutputVars[iy])
    ax[iy].set_ylabel('Prediction, '+InputData.OutputVars[iy])
    
for iSample in range(NSamples):
    yMat = NN.predict(InputDF[InputData.InputVars])
    for iy in range(Ny):
        ax[iy].plot(Output[:,iy], yMat[:,iy], 'ro')

In [15]:
# Plotting Sources vs States

fig  = plt.figure(figsize=(16,12))
ax1  = plt.axes(projection='3d')

fig  = plt.figure(figsize=(16,12))
ax2   = plt.axes(projection='3d')

ax1.plot3D(InputDF[InputData.InputVars[0]], InputDF[InputData.InputVars[1]], OutputDF[InputData.OutputVars[0]], 'ok',  linewidth=2)
ax2.plot3D(InputDF[InputData.InputVars[0]], InputDF[InputData.InputVars[1]], OutputDF[InputData.OutputVars[1]], 'ok',  linewidth=2)

for iSample in range(NSamples):
    yMat = NN.predict(InputDF[InputData.InputVars])
    ax1.plot3D(InputDF[InputData.InputVars[0]], InputDF[InputData.InputVars[1]], yMat[:,0], 'or',  linewidth=2)
    ax2.plot3D(InputDF[InputData.InputVars[0]], InputDF[InputData.InputVars[1]], yMat[:,1], 'or',  linewidth=2)

    ax1.set_xlabel(InputData.InputVars[0])
ax1.set_ylabel(InputData.InputVars[1])
ax1.set_zlabel(InputData.OutputVars[0])

ax2.set_xlabel(InputData.InputVars[0])
ax2.set_ylabel(InputData.InputVars[1])
ax2.set_zlabel(InputData.OutputVars[1])

In [9]:
### Evaluating on Test Data

FileName  = InputData.PathToTestDataFld + '/' + InputData.InputFile
InputDF   = pd.read_csv(FileName, header=0)
Input     = InputDF[InputData.InputVars].to_numpy()
VarNames  = list(InputDF.columns)


FileName  = InputData.PathToTestsDataFld + '/' + InputData.OutputFile
OutputDF  = pd.read_csv(FileName, header=0)
Output    = OutputDF[InputData.OutputVars].to_numpy()

In [16]:
# Plotting Sources vs States

fig  = plt.figure(figsize=(16,12))
ax1  = plt.axes(projection='3d')

fig  = plt.figure(figsize=(16,12))
ax2   = plt.axes(projection='3d')

ax1.plot3D(InputDF[InputData.InputVars[0]], InputDF[InputData.InputVars[1]], OutputDF[InputData.OutputVars[0]], 'ok',  linewidth=2)
ax2.plot3D(InputDF[InputData.InputVars[0]], InputDF[InputData.InputVars[1]], OutputDF[InputData.OutputVars[1]], 'ok',  linewidth=2)

for iSample in range(NSamples):
    yMat = NN.predict(InputDF[InputData.InputVars])
    ax1.plot3D(InputDF[InputData.InputVars[0]], InputDF[InputData.InputVars[1]], yMat[:,0], 'or',  linewidth=2)
    ax2.plot3D(InputDF[InputData.InputVars[0]], InputDF[InputData.InputVars[1]], yMat[:,1], 'or',  linewidth=2)

    ax1.set_xlabel(InputData.InputVars[0])
ax1.set_ylabel(InputData.InputVars[1])
ax1.set_zlabel(InputData.OutputVars[0])

ax2.set_xlabel(InputData.InputVars[0])
ax2.set_ylabel(InputData.InputVars[1])
ax2.set_zlabel(InputData.OutputVars[1])

In [11]:
### Defining Surrogate ODE

def f_NN(t, y):
    #print(t)
    dydt = NN.predict(y[np.newaxis,...])
    return dydt[0,:]

In [12]:
### Evaluating on Test Data

FileName       = TestDir + '/orig_data/RestVecTot.csv'
Data           = pd.read_csv(FileName, header=None)
RestValVecTest = np.log10(Data.to_numpy())

iPC = 0

fig = plt.figure(figsize=(16,12))
for iTest in range(1):

    FileName     = TestDir+'/pc_data_'+str(NVarsRed)+'/PC.csv.'+str(iTest+1)
    Data         = pd.read_csv(FileName, header=0)
    tVec         = Data.to_numpy()[:,0]
    yMat_pca     = Data[Vars].to_numpy()
    y0_pca       = yMat_pca[0,:]
    print('y0_pca = ', y0_pca)
    
    RestVal      = RestValVecTest[iTest]
    
    plt.plot(tVec, yMat_pca[:,iPC], '-k', label='From ODE', lw=2)
    
    output       = solve_ivp( fOrig_NN, tVec[[0,-1]], y0_pca, method='RK45', t_eval=tVec, atol=1.e-8 )
    tNN          = output.t
    yMat_pca_    = output.y.T
    
    plt.plot(tNN, yMat_pca_[:,iPC], ':r', label='From Surrogate Source Term', lw=2)

plt.xscale('log')
plt.xlabel('time [s]')
plt.ylabel('PC_'+str(iPC+1))
plt.legend()

NameError: name 'u' is not defined

In [ ]:
iPC = 0

fig = plt.figure(figsize=(16,12))
plt.plot(tVec, yMat_pca[:,iPC], '-k', label='From ODE',                   lw=2)
plt.plot(tNN, yMat_pca_[:,iPC], ':r', label='From Surrogate Source Term', lw=2)
plt.xscale('log')
plt.xlabel('time [s]')
plt.ylabel('$PC_'+str(iPC+1)+'$')
plt.legend()
#plt.ylim([-2000.0, 2500.])

In [ ]:
FileName = OutputDir+'/pca_'+str(NVarsRed)+'/A.csv'
A        = pd.read_csv(FileName, delimiter=',', header=None).to_numpy()

FileName = OutputDir+'/pca_'+str(NVarsRed)+'/C.csv'
C        = pd.read_csv(FileName, delimiter=',', header=None).to_numpy()

FileName = OutputDir+'/pca_'+str(NVarsRed)+'/D.csv'
D        = pd.read_csv(FileName, delimiter=',', header=None).to_numpy()

In [ ]:
yMat_    = (yMat_pca_.dot(A))*D[:,0] + C[:,0]

In [ ]:
VarOI = 'CO'
for iVar in range(1,len(VarNames)):
    if (VarNames[iVar] == VarOI):
        print('iVar = ', iVar)
        break
        
fig = plt.figure(figsize=(16,12))
for iTest in range(1):

    FileName     = TestDir+'/orig_data/y.csv.'+str(iTest+1)
    Data         = pd.read_csv(FileName, header=0)
    tVec         = Data.to_numpy()[:,0]
    yMat         = Data.to_numpy()[:,1:]
        
    plt.plot(tVec, yMat[:,iVar-1], '-k', label='From ODE',                   lw=2)
    plt.plot(tNN, yMat_[:,iVar-1], ':r', label='From Surrogate Source Term', lw=2)

plt.xscale('log')
plt.xlabel('time [s]')
plt.ylabel(VarOI)
plt.legend()